In [22]:
from __future__ import print_function

import sys
# update your projecty root path before running
sys.path.insert(0, '/home/ajb46717/workDir/projects/nsgaformer')

import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn

import torchvision
from torch.profiler import profile, record_function, ProfilerActivity
from misc import utils

# model imports
from models import macro_genotypes
from models.macro_models import EvoNetwork
import models.micro_genotypes as genotypes
from models.micro_models import ViTNetworkImageNet as Network


In [23]:
arch = "NSGA_ViT"
genotype = eval("genotypes.%s" % arch)
model = Network(512, num_classes=10, layers=6, auxiliary=False, genotype=genotype)
inputs = torch.randn(5, 3, 224, 224)

512 512 512
512 512 512
512 512 512
512 512 512
512 512 512
512 512 512


In [10]:
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
        model(inputs)

STAGE:2023-09-26 11:46:30 3854273:3854273 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-09-26 11:46:30 3854273:3854273 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-09-26 11:46:30 3854273:3854273 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


In [13]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=-1))


--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                            Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                 model_inference         8.06%      20.564ms       100.00%     255.052ms     255.052ms             1  
               aten::convolution         0.52%       1.336ms        61.82%     157.666ms       2.160ms            73  
              aten::_convolution         0.68%       1.728ms        61.29%     156.330ms       2.142ms            73  
        aten::mkldnn_convolution        57.36%     146.294ms        57.88%     147.620ms       2.022ms            73  
                    aten::conv1d         0.19%     479.000us        54.99%     140.257ms       1.948ms            72  
                    aten::linear         0.09%  

In [24]:
gpu_inputs = inputs.cuda()
model = model.cuda()
with profile(activities=[
        ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
    with record_function("model_inference"):
        model(gpu_inputs)

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=-1))

STAGE:2023-09-26 11:56:20 3854273:3854273 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-09-26 11:56:20 3854273:3854273 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-09-26 11:56:20 3854273:3854273 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference        27.57%      10.249ms        99.96%      37.162ms      37.162ms       0.000us         0.00%      10.056ms      10.056ms             1  
                                      aten::convolution         2.25%     837.000us        34.81%      12.942ms     177.288us       0.000us         0.00%       5.955ms      81.575us            73  
         

In [29]:
from torchvision.transforms.v2 import Mixup

ImportError: cannot import name 'Mixup' from 'torchvision.transforms.v2' (/home/ajb46717/anaconda3/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py)

In [26]:
from v2 import MixUp

ModuleNotFoundError: No module named 'v2'